In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection  import train_test_split

## Split Data into Train and Test Datasets

In [2]:
sales = pd.read_csv('kc_house_data.csv')
sales.shape

y = sales['price']
X = sales.drop('price',1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state=0)

# Use spit(), seem we have gotten different X_train and y_train than the one given by the problem. 

train = pd.read_csv('kc_house_train_data.csv')
test = pd.read_csv('kc_house_test_data.csv')

## Learning a multiple regression model
Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features: example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

example_features = train[['sqft_living', 'bedrooms', 'bathrooms']]

regressor.fit(example_features , train['price'])



Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [6]:
# The coefficients
print('Coefficients: \n', regressor.coef_)

print('Intercept: \n', regressor.intercept_)

Coefficients: 
 [   315.40669062 -65081.88711588   6942.16598637]
Intercept: 
 87912.865815


## Making Predictions
In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions.

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [8]:
#prediction the rest set results
y_pred = regressor.predict(example_features)
print(y_pred[0])  # should be 271789.505878

271789.26538


## Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [11]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    
    predictions = model.predict(data)
    
    errors = predictions - outcome
    # Then compute the residuals/errors
    RSS = (errors * errors).sum()
    # Then square and add them up

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [13]:
rss_example_train = get_residual_sum_of_squares(regressor, test[['sqft_living', 'bedrooms', 'bathrooms']], test['price'])

print (rss_example_train) # should be 2.7376153833e+14

273761940583133.97


## Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [15]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:

bedrooms_squared = bedrooms*bedrooms

bed_bath_rooms = bedrooms*bathrooms

log_sqft_living = log(sqft_living)

lat_plus_long = lat + long 

As an example here's the first one:

In [16]:
train['bedrooms_squared'] = train['bedrooms'].apply(lambda x: x**2)

test['bedrooms_squared'] = test['bedrooms'].apply(lambda x: x**2)

In [26]:
# create the remaining 3 features in both TEST and TRAIN data
train['bed_bath_rooms'] = train['bedrooms'] * train['bathrooms']
test['bed_bath_rooms'] = test['bedrooms'] * test['bathrooms']

train['log_sqft_living'] = (train['sqft_living']).apply(log)
test['log_sqft_living'] = (test['sqft_living']).apply(log)

train['lat_plus_long'] = train['lat'] + train['long']  
test['lat_plus_long'] = test['lat'] + test['long']  

Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
bedrooms times bathrooms gives what's called an "interaction" feature. It is large when both of them are large.
Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

## Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)

In [35]:
print('mean value of test_bedrooms_squared: ' + str(test['bedrooms_squared'].mean()))

print('mean value of test_bed_bath_rooms: ' + str(test['bed_bath_rooms'].mean()))

print('mean value of test_log_sqft_living: ' + str(test['log_sqft_living'].mean()))

print('mean value of test_lat_plus_long: ' + str(test['lat_plus_long'].mean()))



mean value of test_bedrooms_squared: 12.4466777015843
mean value of test_bed_bath_rooms: 7.5039016315913925
mean value of test_log_sqft_living: 7.550274679645921
mean value of test_lat_plus_long: -74.65333355403185


## Learning Multiple Models
Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:

Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude

Model 2: add bedrooms*bathrooms

Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [29]:
model_1_features = train[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']]
model_2_features = train[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms']]
model_3_features = train[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms','bedrooms_squared', 'log_sqft_living', 'lat_plus_long']]


Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [30]:
# model 1

regressor1= LinearRegression()

regressor1.fit(model_1_features , train['price'])

# The coefficients
print('Coefficients: \n', regressor1.coef_)

print('Intercept: \n', regressor1.intercept_)

# model 2

regressor2= LinearRegression()

regressor2.fit(model_2_features , train['price'])

# The coefficients
print('Coefficients: \n', regressor2.coef_)

print('Intercept: \n', regressor2.intercept_)

# model 3

regressor3= LinearRegression()

regressor3.fit(model_3_features , train['price'])

# The coefficients
print('Coefficients: \n', regressor3.coef_)

print('Intercept: \n', regressor3.intercept_)

Coefficients: 
 [  3.12258646e+02  -5.95865332e+04   1.57067421e+04   6.58619264e+05
  -3.09374351e+05]
Intercept: 
 -69075726.7926
Coefficients: 
 [  3.06610053e+02  -1.13446368e+05  -7.14613083e+04   6.54844630e+05
  -2.94298969e+05   2.55796520e+04]
Intercept: 
 -66867968.8711
Coefficients: 
 [  5.29422820e+02   3.45142296e+04   6.70607813e+04   5.34085611e+05
  -4.06750711e+05  -8.57050439e+03  -6.78858667e+03  -5.61831484e+05
   1.27334900e+05]
Intercept: 
 -62036084.9861


## Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [33]:
# Rss of model 1

rss_1 = get_residual_sum_of_squares(regressor1, model_1_features, train['price'])

print (rss_1) 

# Rss of model 2

rss_2= get_residual_sum_of_squares(regressor2, model_2_features, train['price'])

print (rss_2) 

# Rss of model 3

rss_3= get_residual_sum_of_squares(regressor3, model_3_features, train['price'])

print (rss_3) 

967879963049549.1
958419635074069.2
903436455050483.5


Now compute the RSS on on TEST data for each of the three models.

In [34]:
# Rss of model 1

rss_test_1 = get_residual_sum_of_squares(regressor1, test[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']], test['price'])

print (rss_test_1) 

# Rss of model 2

rss_test_2= get_residual_sum_of_squares(regressor2, test[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms']], test['price'])

print (rss_test_2) 

# Rss of model 3

rss_test_3= get_residual_sum_of_squares(regressor3, test[['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long','bed_bath_rooms','bedrooms_squared', 'log_sqft_living', 'lat_plus_long']]
, test['price'])

print (rss_test_3) 

225500469795490.3
223377462976466.3
259236319207179.88
